In [1]:
from agents import Runner, Agent, trace, function_tool, WebSearchTool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv
import sendgrid
import asyncio
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

In [4]:
search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [5]:
message = "What is Ansible?"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

Ansible is an open-source automation tool for IT infrastructure management, encompassing configuration management, application deployment, and task automation. It enables users to define system configurations and automate tasks using simple, human-readable YAML files called playbooks. Ansible operates without requiring agents on managed nodes, connecting via SSH or Windows Remote Management, which simplifies deployment and reduces overhead. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Ansible_%28software%29?utm_source=openai))

Developed by Michael DeHaan in 2012 and acquired by Red Hat in 2015, Ansible supports various operating systems, including Linux, Unix-like systems, macOS, and Windows. Its agentless architecture allows for straightforward automation of tasks across diverse environments. Ansible's modular design enables users to extend its functionality, and its idempotent nature ensures that repeated executions yield consistent results, enhancing reliability and predictability in system management. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Ansible_%28software%29?utm_source=openai)) 

In [6]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

In [7]:
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")
    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

In [8]:
planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    output_type=WebSearchPlan
)

In [9]:
message = "What is Ansible?"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To get a general definition and overview of Ansible, including its purpose and primary features.', query='What is Ansible'), WebSearchItem(reason='To understand the common use cases and applications of Ansible in IT and software development.', query='Ansible use cases'), WebSearchItem(reason="To find official resources or documentation that detail Ansible's capabilities and how it works.", query='Ansible official documentation')]


In [10]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("vimalpillai1234@gmail.com")
    to_email = To("vimalpillai47@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [11]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000264C1F69580>, strict_json_schema=True, is_enabled=True)

In [12]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""


In [13]:
email_agent = Agent(
    name="Email Agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4.1-mini"
)

In [14]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

In [15]:
class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(description="The final report")
    follow_up_questions: str = Field(description="Suggested topics to research further")

writer_agent = Agent(name="WriterAgent",
        instructions=INSTRUCTIONS,
        model="gpt-4.1-mini",
        output_type=ReportData
        )

In [16]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished Searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search item: {item.query}\n Reason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [17]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

In [18]:
async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report


In [19]:
query = "What is Ansible?"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Finished. Please check email")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished Searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Finished. Please check email
